# The use and misuse of pathway enrichment analysis in metabolomics

#### This notebook contains the code to run the following simulations:
- Background list
- Background list size reduction
- DA metabolite list size

#### The following scripts are required to be able to run the code. They should be saved in the same folder as this notebook. 
- process_datasets.py
- utils.py

All packages required are listed in the requirements.txt file. 

In [1]:
import utils
import process_datasets
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

ModuleNotFoundError: No module named 'molmass'